In [1]:
from bread.vis import *
from bread.data import Features, SegmentationFile, Microscopy
from bread.algo.lineage import LineageGuesser, LineageGuesserNN, LineageGuesserML, LineageGuesserNearestCell, accuracy
from bread.data import Lineage

import pandas as pd
import pathlib
import os

# test for colony 0 to 5 individually


In [4]:
args = {'fov': 0, 'bud_distance_max': 12, 'num_frames_refractory': 0, 'num_frames': 8}
data_path = '../../../data/'
model_path = '../algo/lineage/saved_models/best_model_with_fake_candid_thresh12_frame_num8_normalized_False.pth'
lineage_guesses = []

# find predictions for each colony
for colony in range(0,6):
    output_file = os.path.join(data_path, f'colony00{colony}_lineage_guess.csv')
    segmentation_file = os.path.join(data_path, f'colony00{colony}_segmentation.h5')
    segmentation = SegmentationFile.from_h5(segmentation_file).get_segmentation('FOV'+str(args['fov']))
    guesser = LineageGuesserNN(
        saved_model=model_path,
        segmentation=segmentation,
        nn_threshold=args['bud_distance_max'],
        num_frames_refractory=args['num_frames_refractory'],
        num_frames=args['num_frames'],
    )

    lineage_guess: Lineage = guesser.guess_lineage()
    lineage_guesses.append(lineage_guess)
    lineage_guess.save_csv(output_file)

/Users/farzanehwork/Documents/codes/bread/src/bread/algo/lineage/_lineage.py:959: NotEnoughFramesWarning: Not enough frames in the movie (bud #76 at frame #174), requested 8, but only 7 remaining.
  warnings.warn(NotEnoughFramesWarning(bud_id, time_id, self.num_frames, len(frame_range)))
/Users/farzanehwork/Documents/codes/bread/src/bread/algo/lineage/_lineage.py:959: NotEnoughFramesWarning: Not enough frames in the movie (bud #77 at frame #175), requested 8, but only 6 remaining.
  warnings.warn(NotEnoughFramesWarning(bud_id, time_id, self.num_frames, len(frame_range)))
/Users/farzanehwork/Documents/codes/bread/src/bread/algo/lineage/_lineage.py:959: NotEnoughFramesWarning: Not enough frames in the movie (bud #78 at frame #175), requested 8, but only 6 remaining.
  warnings.warn(NotEnoughFramesWarning(bud_id, time_id, self.num_frames, len(frame_range)))
/Users/farzanehwork/Documents/codes/bread/src/bread/algo/lineage/_lineage.py:959: NotEnoughFramesWarning: Not enough frames in the mo

In [7]:
# find accuracy for each colony
for colony, lin_guess in enumerate(lineage_guesses):
    lineage_gt_path = os.path.join(data_path, f'colony00{colony}_lineage.csv')
    lin_truth = Lineage.from_csv(lineage_gt_path)
    acc = accuracy(lin_truth, lin_guess, strict=False)
    print('accuracy for colony {}: {}'.format(colony , acc))
    

accuracy for colony 0: 0.918918918918919
accuracy for colony 1: 0.8780487804878049
accuracy for colony 2: 0.8414634146341463
accuracy for colony 3: 0.8543689320388349
accuracy for colony 4: 0.9333333333333333


# test on external dataset


In [15]:
model_path = '/home/farzaneh/Documents/Bread/bread/src/bread/algo/lineage/saved_models/best_model_with_fake_candid_thresh12_frame_num8_normalized_True.pth'


In [16]:
colony = 10

# test the model on the external subset
external_subset_lineage_GT_path_edited = '/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset/lineage/lineage_budlum_edited_by_code.csv'
external_subset_segmentation_path = '/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset/segmentation2/stacked/mask_stack _edited.h5'
segmentation = SegmentationFile.from_h5(external_subset_segmentation_path).get_segmentation('FOV'+str(args['fov']))
output_file = '/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset/lineage/lineage_guess_nn.csv'

guesser = LineageGuesserNN(
    saved_model=model_path,
    segmentation=segmentation,
    nn_threshold=args['bud_distance_max'],
    num_frames_refractory=args['num_frames_refractory'],
    num_frames=args['num_frames'],
)

lineage_guess: Lineage = guesser.guess_lineage()
lineage_guess.save_csv(output_file)


In [17]:

external_subset_lineage_GT_path_edited = '/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset/lineage/lineage_budlum_edited_by_code.csv'
lin_truth = Lineage.from_csv(external_subset_lineage_GT_path_edited)
acc = accuracy(lin_truth, lineage_guess, strict=False)
print('accuracy for colony {}: {}'.format(colony , acc))


accuracy for colony 10: 0.8166089965397924
